Scrap websites mentioned in file to collect food categories. It saves scrapped data after each page. 

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import time
from tqdm import tqdm

## Read file

You have to specified number of rows to read from a file - check first how many rows are there in the scv file.
Without 'nrows' parameter an exception occures.

In [48]:
df = pd.read_csv('food_data_excerpt/food_data_excerpt.csv', sep=',', encoding='utf8', nrows=336936)#, engine='python'

In [42]:
df[40:50]

,country,country_region,zip,city,date,target_link,origin_link,host_type,recipe_id
40,DE,13,4651,Bad_Lausick,2013-03-01 00:00:58,/rezept/363977/Entenbraten-klassisch.html,http://www.google.de/url?sa=t&rct=j&q=&esrc=s&...,Mozilla/5.0 (Windows NT 5.1; rv:19.0) Gecko/20...,363977
41,DE,9,66564,Ottweiler,2013-03-01 00:01:00,/rezept/413547/Coq-au-Vin.html,http://www.kochbar.de/rezepte/coq-au-vin.html,Mozilla/5.0 (Windows NT 5.1; rv:19.0) Gecko/20...,413547
42,DE,11,-1,Brandenburg,2013-03-01 00:01:01,/rezept/397242/Spanferkelbraten-NT-mit-frische...,-,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,397242
43,DE,6,-1,Hanover,2013-03-01 00:01:01,/rezept/450876/Gemuesenudeln.html,http://www.kochbar.de/rezepte/alle-rezepte.htm...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:19.0) G...,450876
44,DE,5,-1,Kassel,2013-03-01 00:01:01,/rezept/38466/Schneller-Himbeerkuchen.html,http://www.kochbar.de/rezepte/himbeertorte.htm...,Mozilla/5.0 (iPad; CPU OS 6_0_1 like Mac OS X)...,38466
45,DE,7,-1,Essen,2013-03-01 00:01:02,/rezept/317291/Bildrezept-Eiweiss-Schoko-Kuche...,https://www.google.com/,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,317291
46,DE,8,56076,Koblenz,2013-03-01 00:01:02,/rezept/458991/Pistazien-Frischkaese-Brownies....,http://www.kochbar.de/kochen/Kanadisch-kochen-...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; r...,458991
47,DE,2,-1,Neusäß,2013-03-01 00:01:04,/rezept/269164/Fenchelgemuese-zu-Rotbarsch-mit...,http://www.kochbar.de/rezepte/fenchelgem%C3%BC...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:19.0) G...,269164
48,DE,16,14165,Berlin,2013-03-01 00:01:09,/rezept/209594/Haehnchenbrustfilet-im-Speckman...,http://www.kochbar.de/rezepte/h%C3%A4hnchenbru...,Mozilla/5.0 (Windows NT 6.0; rv:19.0) Gecko/20...,209594
49,CH,5,-1,Bern,2013-03-01 00:01:10,/rezept/421887/Fitness-Teller.html,http://www.google.ch/url?sa=t&rct=j&q=fitnesst...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,421887


## Collect data

In [4]:
def find_comment(soup, wanted_comment='REZEPT-KATEGORIEN'):
    found = False

    comments = soup.find_all(string=lambda text:isinstance(text, Comment))
    i = 0
    while not found and i < len(comments):
        node = comments[i]
        if node.strip() == wanted_comment:
            #print('comment found')
            found = True

        i+=1
    return node

In [5]:
def find_categories(soup, wanted_class = 'rtli-pb-small'):
    data = []
    node = find_comment(soup)
    startOfItems, endOfItems = False, False
    
    while not endOfItems and node.find_next_sibling():
        node = node.find_next_sibling()

        if node['class'][0] == wanted_class:
            startOfItems = True

            if node.findChild().name == 'a':
                data.append(node.a.text)
            elif node.findChild().name == 'span':
                data.append(node.span.text)

        elif startOfItems:
            endOfItems = True
    
    return data

In [6]:
domain = 'http://www.kochbar.de'

Remember to specify with rows to process. If 'all_rows' is set to True, 'start_index' and 'end_index' is not taken into account.

In [92]:
start_index, end_index = 40, 50
all_rows = True

In [105]:
start = time.time()

error_num = 0 
df_new = pd.DataFrame(columns=['date', 'recipe_id', 'categories'])
f= open('my_csv.csv','w')
df_new.to_csv(f, index=None, header=True, encoding='utf-8', sep=',')
f.close()

nrows = 336936
df = pd.read_csv('food_data_excerpt/food_data_excerpt.csv', sep=',', encoding='utf-8', nrows=nrows)#, engine='python'

f= open('my_csv.csv', 'a')

if not all_rows:
    df = df[start_index:end_index]
else:
    start_index = 0

with tqdm(total=nrows) as pbar:
    for i in range(len(df)): 
        #print(domain + url)
        url = df['target_link'][start_index + i]
        response = requests.get(domain + url)

        if response.url != 'https://www.kochbar.de/rezepte/' and response.status_code == 200:
            #print('Request status ok!')
            soup = BeautifulSoup(response.content.decode('utf-8','ignore'), "lxml")
            categories = find_categories(soup)
            #print('categories:', categories, '\n')
            #data.append(categories)
            s = pd.DataFrame({'d': str(df['date'][start_index + i]), 'id': df['recipe_id'][start_index + i], 'c': str(categories)}, index=[0])
        elif response.status_code == 200:
            #print('This webpage is probably no longer available!')
            #data.append(None)
            s = pd.DataFrame({'d': df['date'][start_index + i], 'id': df['recipe_id'][start_index + i], 'c': str([])}, index=[0])

        else:
            #print('Not correct request status: ', response.status_code)
            #data.append(None)
            s = pd.DataFrame({'d': df['date'][start_index + i], 'id': df['recipe_id'][start_index + i], 'c': str([])}, index=[0])
        try:
            s.to_csv(f, index=None, header=False, encoding='utf-8', sep=',')
        except UnicodeEncodeError:
            print('UnicodeEncodeError')
            error_num+=1
        pbar.update(1)

f.close() 
print(time.time() - start)
print('errors:', error_num)

  0%|          | 116/336936 [00:33<20:30:47,  4.56it/s]

UnicodeEncodeError


  0%|          | 130/336936 [00:44<81:03:14,  1.15it/s]

UnicodeEncodeError


  0%|          | 138/336936 [00:51<68:02:17,  1.38it/s] 

...

UnicodeEncodeError


  1%|          | 1972/336936 [24:21<45:36:38,  2.04it/s] 

UnicodeEncodeError


  1%|          | 2032/336936 [24:55<49:39:52,  1.87it/s]


KeyboardInterrupt: 

In [ ]:
f.close() 

## Check saved data

In [4]:
pd.read_csv('my_csv.csv', encoding='utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xdc in position 11: invalid continuation byte